# Libraries and Data (TURN ON GPU)

In [ ]:
# Install the darts library
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.7/884.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.5/223.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6

In [ ]:
# Standard Libraries
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import month_plot, quarter_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np
from sklearn.model_selection import ParameterGrid

# Darts functions
from darts.timeseries import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel

In [ ]:
# Load the CSV data into a NumPy array
df = pd.read_csv("Hourly-train.csv", header = None).values.squeeze()
df

array([['V1', 'V2', 'V3', ..., 'V959', 'V960', 'V961'],
       ['H1', '605', '586', ..., nan, nan, nan],
       ['H2', '3124', '2990', ..., nan, nan, nan],
       ...,
       ['H412', '19', '30', ..., '73', '39', '36'],
       ['H413', '21', '16', ..., '53', '38', '46'],
       ['H414', '15', '13', ..., '35', '26', '17']], dtype=object)

In [ ]:
# Exclude the first row and first column
df = df[1:, 1:].astype(np.float64)
df

array([[ 605.,  586.,  586., ...,   nan,   nan,   nan],
       [3124., 2990., 2862., ...,   nan,   nan,   nan],
       [1828., 1806., 1897., ...,   nan,   nan,   nan],
       ...,
       [  19.,   30.,   12., ...,   73.,   39.,   36.],
       [  21.,   16.,   17., ...,   53.,   38.,   46.],
       [  15.,   13.,   13., ...,   35.,   26.,   17.]])

In [ ]:
# Transpose the data
df = df.T
df

array([[ 605., 3124., 1828., ...,   19.,   21.,   15.],
       [ 586., 2990., 1806., ...,   30.,   16.,   13.],
       [ 586., 2862., 1897., ...,   12.,   17.,   13.],
       ...,
       [  nan,   nan,   nan, ...,   73.,   53.,   35.],
       [  nan,   nan,   nan, ...,   39.,   38.,   26.],
       [  nan,   nan,   nan, ...,   36.,   46.,   17.]])

In [ ]:
# Checking which rows have NaN values
na_rows = np.isnan(df).any(axis = 1)
np.where(na_rows)

(array([700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712,
        713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725,
        726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738,
        739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751,
        752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764,
        765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777,
        778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790,
        791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803,
        804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816,
        817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829,
        830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842,
        843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855,
        856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868,
        869, 870, 871, 872, 873, 874, 875, 876, 877

In [ ]:
# Remove rows with NaN values
df = df[~np.any(np.isnan(df), axis = 1)]
df

array([[ 605., 3124., 1828., ...,   19.,   21.,   15.],
       [ 586., 2990., 1806., ...,   30.,   16.,   13.],
       [ 586., 2862., 1897., ...,   12.,   17.,   13.],
       ...,
       [ 752., 3460., 1581., ...,   18.,   17.,   13.],
       [ 739., 3248., 1570., ...,   13.,   16.,   17.],
       [ 684., 3105., 1518., ...,   17.,   15.,   21.]])

In [ ]:
# Check how many rows we have
len(df)

700

In [ ]:
# Convert the array into pd.Dataframe
df = pd.DataFrame(df)

In [ ]:
# Because we don't have time information, we will set dummy time

# Set a dummy start date for time information
start_date = pd.Timestamp("2000-01-01 00:00:00")

# Create a TimeSeries object with dummy time information
series = TimeSeries.from_times_and_values(
    pd.date_range(start = start_date, periods = len(df), freq = "H"),
    df.values
)

# Display
series

<TimeSeries (DataArray) (time: 700, component: 414, sample: 1)>
array([[[ 605.],
        [3124.],
        [1828.],
        ...,
        [  19.],
        [  21.],
        [  15.]],

       [[ 586.],
        [2990.],
        [1806.],
        ...,
        [  30.],
        [  16.],
        [  13.]],

       [[ 586.],
        [2862.],
        [1897.],
        ...,
...
        ...,
        [  18.],
        [  17.],
        [  13.]],

       [[ 739.],
        [3248.],
        [1570.],
        ...,
        [  13.],
        [  16.],
        [  17.]],

       [[ 684.],
        [3105.],
        [1518.],
        ...,
        [  17.],
        [  15.],
        [  21.]]])
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01 ... 2000-01-30T03:00:00
  * component  (component) <U3 '0' '1' '2' '3' '4' ... '410' '411' '412' '413'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

# Exploratory Data Analysis

In [ ]:
# Simple plots
plt.figure(figsize = (18,6))
df.plot()
plt.show()

In [ ]:
## Daily seasonality

# Convert TimeSeries to a DataFrame with time information
df_with_time = series.pd_dataframe()

# Compute hourly means for daily seasonality
hourly_means = df_with_time.groupby(df_with_time.index.hour).mean()

# Plot the average values for the first 10 columns
plt.figure(figsize = (18,6))
plt.plot(hourly_means.iloc[:,0:10], marker = "o")
plt.grid()
plt.title("Hourly Seasonality Plot")
plt.xlabel("Hours")
plt.ylabel("Average per hour")
plt.xticks(range(24))
plt.show()

In [ ]:
# Standardize the hourly means
hourly_means_std = (hourly_means - np.mean(hourly_means, axis = 0)) / np.std(hourly_means, axis = 0)

# Plot the standardized hourly means for the first 100 columns
plt.figure(figsize = (18,6))
plt.plot(hourly_means_std.iloc[:,0:100], marker = "o")
plt.grid()
plt.title("Hourly Seasonality Plot")
plt.xlabel("Hours")
plt.ylabel("Standardized Values")
plt.xticks(range(24))
plt.show()

In [ ]:
## Weekly Seasonality

# Compute daily means for weekly seasonality
daily_means = df_with_time.groupby(df_with_time.index.dayofweek).mean()
# Standardize the daily means
daily_means_std = (daily_means - np.mean(daily_means, axis = 0)) / np.std(daily_means, axis = 0)

# Plot the standardized daily means for the first 100 columns
plt.figure(figsize = (18,6))
plt.plot(daily_means_std.iloc[:,0:100], marker = "o")
plt.grid()
plt.title("Daily Seasonality Plot")
plt.xlabel("Day of week")
plt.ylabel("Average per weekday")
plt.xticks(range(7))
plt.show()

# Prepare for LSTM

In [ ]:
# Create time series instances

# Month
hour_series = datetime_attribute_timeseries(series,
                                             attribute = "hour",
                                             one_hot = True)

# Weekday
weekday_series = datetime_attribute_timeseries(series,
                                             attribute = "weekday",
                                             one_hot = True)

# Display the weekday series
weekday_series

<TimeSeries (DataArray) (time: 700, component: 7, sample: 1)>
array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [1.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [1.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
...
        ...,
        [0.],
        [0.],
        [1.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]]])
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01 ... 2000-01-30T03:00:00
  * component  (component) object 'weekday_0' 'weekday_1' ... 'weekday_6'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [ ]:
# Import the Scaler class and initialize it
scaler1 = Scaler()
#scaler2 = Scaler()

In [ ]:
# Apply the scaler to the time series
series_transformed = scaler1.fit_transform(series)
series_transformed

<TimeSeries (DataArray) (time: 700, component: 414, sample: 1)>
array([[[0.44367418],
        [0.65096192],
        [0.93129062],
        ...,
        [0.02140673],
        [0.09345794],
        [0.00693241]],

       [[0.41074523],
        [0.598351  ],
        [0.91086351],
        ...,
        [0.05504587],
        [0.04672897],
        [0.0034662 ]],

       [[0.41074523],
        [0.5480958 ],
        [0.99535747],
        ...,
...
        ...,
        [0.01834862],
        [0.05607477],
        [0.0034662 ]],

       [[0.67590988],
        [0.69964664],
        [0.6917363 ],
        ...,
        [0.0030581 ],
        [0.04672897],
        [0.01039861]],

       [[0.58058925],
        [0.64350216],
        [0.64345404],
        ...,
        [0.01529052],
        [0.03738318],
        [0.01733102]]])
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01 ... 2000-01-30T03:00:00
  * component  (component) <U3 '0' '1' '2' '3' '4' ... '410' '411' '412' '413'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [ ]:
# Stack the weekday and hour covariates into a single TimeSeries object
covariates_transformed = weekday_series.stack(hour_series)
covariates_transformed

<TimeSeries (DataArray) (time: 700, component: 31, sample: 1)>
array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
...
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])
Coordinates:
  * time       (time) datetime64[ns] 2000-01-01 ... 2000-01-30T03:00:00
  * component  (component) object 'weekday_0' 'weekday_1' ... 'hour_23'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

#LSTM model

In [ ]:
# Define the input chunk length, forecasting horizon, and training length
forecasting_horizon = 48
input_chunk_length = 72
training_length = forecasting_horizon + input_chunk_length

In [ ]:
# LSTM model
#https://unit8co.github.io/darts/generated_api/darts.models.forecasting.rnn_model.html

model = RNNModel(model = "LSTM",                             # Specify LSTM model
                 hidden_dim = 20,                            # Number of hidden units
                 n_rnn_layers = 2,                           # Number of RNN layers
                 dropout = 0.1,                              # Dropout rate
                 n_epochs = 10,                              # Number of training epochs
                 optimizer_kwargs = {"lr": 0.003},           # Learning rate for the optimizer
                 random_state = 1502,                        # Random seed for reproducibility
                 training_length = training_length,          # Length of training data
                 input_chunk_length = input_chunk_length,    # Length of input chunks
                 pl_trainer_kwargs = {"accelerator": "cpu"}  # Use CPU for training
                # pl_trainer_kwargs = {"accelerator": "gpu",
                #                       "devices": [0]}  # Specify GPU device
                 )

# Fit the model with the transformed time series data and covariates
model.fit(series_transformed, future_covariates = covariates_transformed)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | criterion     | MSELoss          | 0      | train
1 | train_metrics | MetricCollection | 0      | train
2 | val_metrics   | MetricCollection | 0      | train
3 | rnn           | LSTM             | 40.7 K | train
4 | V             | Linear           | 8.7 K  | train
-----------------------------------------------------------
49.4 K    Trainable params
0         Non-trainable params
49.4 K    Total params
0.198     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


RNNModel(model=LSTM, hidden_dim=20, n_rnn_layers=2, dropout=0.1, training_length=120, n_epochs=10, optimizer_kwargs={'lr': 0.003}, random_state=1502, input_chunk_length=72, pl_trainer_kwargs={'accelerator': 'cpu'})

# Cross Validation

In [ ]:
# Perform cross-validation using a rolling forecasting window
cv = model.historical_forecasts(series = series_transformed,
                                future_covariates = covariates_transformed,
                                start = df.shape[0] - 200,               # Start point for forecasting window
                                forecast_horizon = forecasting_horizon,  # Forecast horizon
                                stride = 24,                             # Stride for rolling window
                                retrain = True,                          # Retrain model at each step
                                last_points_only = False)                # Use all points for forecasting


In [ ]:
# Initialize a list to store RMSE values for cross-validation
rmse_cv = []

# Iterate over each forecast result from cross-validation
for i in range(len(cv)):

  # Convert the forecasted values back to the original scale
  predictions = TimeSeries.pd_dataframe(scaler1.inverse_transform(cv[i]))
  # Convert original time series to DataFrame
  series_pandas = TimeSeries.pd_dataframe(series)

  # Determine the range of dates for actual values
  start = predictions.index.min()
  end = predictions.index.max()
  # Extract the actual values from the original dataset
  actuals = series_pandas[start:end]

  # Calculate the RMSE for the current cross-validation fold
  error_cv = np.sqrt(mean_squared_error(actuals, predictions))
  # Append the RMSE to the list
  rmse_cv.append(error_cv)

# Print the mean RMSE across all cross-validation folds
print(np.mean(rmse_cv))

3195.785397647874


In [ ]:
# Visualize the forecasts from each cross-validation fold
for i in range(len(cv)):
  plt.figure(figsize = (18,6))

  # Plot the predicted values for the current fold
  scaler1.inverse_transform(cv[i]).plot(label = "Predicted",
                                        max_nr_components=4)
  # Plot the actual values for comparison
  series.plot(label = "actuals",
              max_nr_components = 4)

  # Add legend and title
  plt.legend()
  plt.title(f"M4 fitting: validation for fold {i}")
  plt.show()

# Parameter Tuning

In [ ]:
# Define the parameter grid for tuning
param_grid = {'n_rnn_layers': [2,4],
              'hidden_dim': [10,20],
              'dropout': [0.1, 0.2],
              'n_epochs': [10, 20],
              'lr': [0.003],
              'training_length': [120],
              'input_chunk_length': [72]}

# Generate all combinations of the parameters
grid = ParameterGrid(param_grid)
# Get the total number of parameter combinations
len(list(grid))

16

In [ ]:
# Initialize a list to store RMSE values for each parameter combination
rmse = []

# Iterate over each parameter combination in the grid
for params in grid:
  # Build the LSTM model with the current set of parameters
  model = RNNModel(model = "LSTM",                                       # Specify LSTM model
                    hidden_dim = params['hidden_dim'],                   # Number of hidden units
                    n_rnn_layers = params['n_rnn_layers'],               # Number of RNN layers
                    dropout = params['dropout'],                         # Dropout rate
                    n_epochs = params['n_epochs'],                       # Number of training epochs
                    optimizer_kwargs = {"lr": params['lr']},             # Learning rate for the optimizer
                    random_state = 1502,                                 # Random seed for reproducibility
                    training_length = params['training_length'],         # Length of training data
                    input_chunk_length = params['input_chunk_length'],   # Length of input chunks
                    # pl_trainer_kwargs = {"accelerator": "cpu"}
                    pl_trainer_kwargs = {"accelerator": "gpu",           # Use GPU for training
                                        "devices": [0]}
                    )

  # Fit the model with the transformed time series data and covariates
  model.fit(series_transformed, future_covariates = covariates_transformed)

  # Perform cross-validation with a rolling forecasting window
  cv = model.historical_forecasts(series = series_transformed,
                                  future_covariates = covariates_transformed,
                                  start = df.shape[0] - 200,                 # Start point for the forecasting window
                                  forecast_horizon = forecasting_horizon,    # Forecast horizon
                                  stride = 24,                               # Stride for the rolling window
                                  retrain = True,                            # Retrain model at each step
                                  last_points_only = False)                  # Use all points for forecasting

  # Initialize a list to store RMSE values for cross-validation
  rmse_cv = []

  # Iterate over each forecast result from cross-validation
  for i in range(len(cv)):

    # Convert forecasted values back to the original scale
    predictions = TimeSeries.pd_dataframe(scaler1.inverse_transform(cv[i]))
    # Convert original time series to DataFrame
    series_pandas = TimeSeries.pd_dataframe(series)

    # Determine the range of dates for actual values
    start = predictions.index.min()
    end = predictions.index.max()

    # Extract the actual values from the original dataset
    actuals = series_pandas[start:end]

    # Calculate the RMSE for the current cross-validation fold
    error_cv = np.sqrt(mean_squared_error(actuals, predictions))
    # Append the RMSE to the list
    rmse_cv.append(error_cv)

  # Calculate and store the mean RMSE for the current parameter combination
  error = np.mean(rmse_cv)
  rmse.append(error)

In [ ]:
# Retrieve the best params and put in in a dataframe
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

,dropout,hidden_dim,input_chunk_length,lr,n_epochs,n_rnn_layers,training_length,rmse
0,0.1,10,72,0.003,10,2,120,5898.301699
1,0.1,10,72,0.003,10,4,120,5953.740363
2,0.1,10,72,0.003,20,2,120,2461.495631
3,0.1,10,72,0.003,20,4,120,3427.892543
4,0.1,20,72,0.003,10,2,120,3206.301438
5,0.1,20,72,0.003,10,4,120,4886.829620
6,0.1,20,72,0.003,20,2,120,3020.420884
7,0.1,20,72,0.003,20,4,120,2925.460223
8,0.2,10,72,0.003,10,2,120,5738.578239
9,0.2,10,72,0.003,10,4,120,6196.096256


In [ ]:
# Identify the parameter set(s) with the lowest RMSE and export it to a CSV file
best_params = tuning_results[tuning_results['rmse'] == tuning_results['rmse'].min()]
best_params = best_params.iloc[0,:]
best_params.to_csv("best_params_round1.csv")

# Parameter Round 2

In [ ]:
# Load the best parameters from the CSV file
params_round1 = pd.read_csv("best_params_round1.csv", index_col = 0)
params_round1

,10
dropout,0.200000
hidden_dim,10.000000
input_chunk_length,72.000000
lr,0.003000
n_epochs,20.000000
n_rnn_layers,2.000000
training_length,120.000000
rmse,2432.279892


In [ ]:
# Isolate and convert the three parameters from the best parameter set
n_rnn_layers = int(params_round1.loc["n_rnn_layers"][0])  # Extract and convert the number of RNN layers
hidden_dim = int(params_round1.loc["hidden_dim"][0])      # Extract and convert the number of hidden units
dropout = params_round1.loc["dropout"][0]                 # Extract the dropout rate
dropout  # Display the dropout rate

0.2

In [ ]:
# Define the parameter grid for tuning
param_grid = {'n_rnn_layers': [n_rnn_layers],
              'hidden_dim': [hidden_dim],
              'dropout': [dropout],
              'n_epochs': [10,20 ],
              'lr': [0.0005, 0.003],
              'training_length': [120, 150],
              'input_chunk_length': [72,100]}

# Generate all combinations of the parameters
grid = ParameterGrid(param_grid)
# Get the total number of parameter combinations
len(list(grid))

16

In [ ]:
# Initialize a list to store RMSE values for each parameter combination
rmse = []

# Iterate over each parameter combination in the grid
for params in grid:
  # Build the LSTM model with the current set of parameters
  model = RNNModel(model = "LSTM",
                  hidden_dim = params['hidden_dim'],                  # Set the number of hidden units
                  n_rnn_layers = params['n_rnn_layers'],              # Set the number of RNN layers
                  dropout = params['dropout'],                        # Set the dropout rate
                  n_epochs = params['n_epochs'],                      # Set the number of training epochs
                  optimizer_kwargs = {"lr": params['lr']},            # Set the learning rate
                  random_state = 1502,                                # Set random seed for reproducibility
                  training_length = params['training_length'],        # Set the training length
                  input_chunk_length = params['input_chunk_length'],  # Set the length of input chunks
                 # pl_trainer_kwargs = {"accelerator": "cpu"}
                  pl_trainer_kwargs = {"accelerator": "gpu",
                                       "devices": [0]}                # Specify training on GPU
                  )

  # Fit the model with the transformed data and covariates
  model.fit(series_transformed, future_covariates = covariates_transformed)

  # Perform cross-validation using a rolling forecasting window
  cv = model.historical_forecasts(series = series_transformed,
                                  future_covariates = covariates_transformed,
                                  start = df.shape[0] - 200,
                                  forecast_horizon = forecasting_horizon,  # Forecast horizon
                                  stride = 24,                             # Stride for the rolling window
                                  retrain = True,                          # Retrain the model at each step
                                  last_points_only = False)                # Use all data points for forecasting

  # Initialize a list to store RMSE values for cross-validation
  rmse_cv = []

  # Iterate over each forecast result from cross-validation
  for i in range(len(cv)):

    # Convert forecasted values back to the original scale
    predictions = TimeSeries.pd_dataframe(scaler1.inverse_transform(cv[i]))
    # Convert original time series to DataFrame
    series_pandas = TimeSeries.pd_dataframe(series)

    # Determine the range of dates for actual values
    start = predictions.index.min()
    end = predictions.index.max()

    # Extract the actual values from the original dataset
    actuals = series_pandas[start:end]

    # Calculate the RMSE for the current cross-validation fold
    error_cv = np.sqrt(mean_squared_error(actuals, predictions))
    # Append the RMSE to the list
    rmse_cv.append(error_cv)

  # Calculate and store the mean RMSE for the current parameter combination
  error = np.mean(rmse_cv)
  rmse.append(error)

In [ ]:
# Retrieve the best params and put in in a dataframe
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

,dropout,hidden_dim,input_chunk_length,lr,n_epochs,n_rnn_layers,training_length,rmse
0,0.2,10,72,0.0005,10,2,120,7216.083109
1,0.2,10,72,0.0005,10,2,150,7387.608361
2,0.2,10,72,0.0005,20,2,120,6547.292457
3,0.2,10,72,0.0005,20,2,150,6545.427988
4,0.2,10,72,0.0030,10,2,120,5738.578239
5,0.2,10,72,0.0030,10,2,150,5719.745911
6,0.2,10,72,0.0030,20,2,120,2432.279892
7,0.2,10,72,0.0030,20,2,150,2493.410201
8,0.2,10,100,0.0005,10,2,120,7216.083109
9,0.2,10,100,0.0005,10,2,150,7387.608361


In [ ]:
# Identify the parameter set(s) with the lowest RMSE and export it to a CSV file
best_params = tuning_results[tuning_results['rmse'] == tuning_results['rmse'].min()]
best_params = best_params.iloc[0,:]
best_params.to_csv("best_params_round2.csv")

# Predict the future

In [ ]:
# Load the best parameters from the CSV file
params_round2 = pd.read_csv("best_params_round2.csv", index_col = 0)
params_round2

,14
dropout,0.200000
hidden_dim,10.000000
input_chunk_length,100.000000
lr,0.003000
n_epochs,20.000000
n_rnn_layers,2.000000
training_length,120.000000
rmse,2432.279892


In [ ]:
# Isolate and convert all parameters from the best parameter set

n_rnn_layers = int(params_round2.loc["n_rnn_layers"][0])
hidden_dim = int(params_round2.loc["hidden_dim"][0])
dropout = params_round2.loc["dropout"][0]
input_chunk_length = int(params_round2.loc["input_chunk_length"][0])
lr = params_round2.loc["lr"][0]
n_epochs = int(params_round2.loc["n_epochs"][0])
training_length = int(params_round2.loc["training_length"][0])

In [ ]:
# Generate future time index starting from one hour after the end of the series
future_time_index = pd.date_range(start = series.end_time() + pd.Timedelta(hours = 1),
                                  periods = 48,   # Number of periods to forecast
                                  freq = "H")

# Create a TimeSeries object for the hour attribute
hour_future = datetime_attribute_timeseries(future_time_index,
                                             attribute = "hour",
                                             one_hot = True)

# Create a TimeSeries object for the weekday attribute
weekday_future = datetime_attribute_timeseries(future_time_index,
                                             attribute = "weekday",
                                             one_hot = True)

In [ ]:
# Stack the future covariates (hour and weekday) into a single TimeSeries object
future_covariates = hour_future.stack(weekday_future)
# Append the future covariates to the existing transformed covariates
covariates_transformed = covariates_transformed.append(future_covariates)

In [ ]:
# LSTM model
#https://unit8co.github.io/darts/generated_api/darts.models.forecasting.rnn_model.html

model = RNNModel(model = "LSTM",                              # Specify the LSTM model
                 hidden_dim = hidden_dim,                     # Number of hidden units
                 n_rnn_layers = n_rnn_layers,                 # Number of RNN layers
                 dropout = dropout,                           # Dropout rate
                 n_epochs = n_epochs,                         # Number of training epochs
                 optimizer_kwargs = {"lr": lr},               # Learning rate for the optimizer
                 random_state = 1502,                         # Random seed for reproducibility
                 training_length = training_length,           # Length of training data
                 input_chunk_length = input_chunk_length,     # Length of input chunks
                 pl_trainer_kwargs = {"accelerator": "cpu"}   # Use CPU for training
                # pl_trainer_kwargs = {"accelerator": "gpu",
                #                      "devices": [0]}
                 )

# Fit the model with the transformed time series data and covariates
model.fit(series_transformed, future_covariates = covariates_transformed)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | criterion     | MSELoss          | 0      | train
1 | train_metrics | MetricCollection | 0      | train
2 | val_metrics   | MetricCollection | 0      | train
3 | rnn           | LSTM             | 19.2 K | train
4 | V             | Linear           | 4.6 K  | train
-----------------------------------------------------------
23.7 K    Trainable params
0         Non-trainable params
23.7 K    Total params
0.095     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


RNNModel(model=LSTM, hidden_dim=10, n_rnn_layers=2, dropout=0.2, training_length=120, n_epochs=20, optimizer_kwargs={'lr': 0.003}, random_state=1502, input_chunk_length=100, pl_trainer_kwargs={'accelerator': 'cpu'})

In [ ]:
# Predict the future values for the next 48 periods
predictions = model.predict(n = 48,
                            future_covariates = covariates_transformed)

# Convert the predicted values back to the original scale
predictions = TimeSeries.pd_dataframe(scaler1.inverse_transform(predictions))

# Display
predictions